<h1><b style="color: yellow;">Assignment 1</b></h1>


## <u> Imports

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_chroma import Chroma
from langchain_core.documents import Document
from pathlib import Path
from langchain_core.output_parsers import StrOutputParser

In [ ]:
llm = Ollama(model = 'mistral')

# <u> RAG

## Splitting text

In [ ]:
with open('speech.txt','r',encoding='utf-8') as f:
    speech_text = f.read()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=40) # I kept the over lapping a little high for our task but i wanted to context to flow
texts = text_splitter.split_text(speech_text)
texts


['The real remedy is to destroy the belief in the sanctity of the shastras. How do you expect to',
 'of the shastras. How do you expect to succeed if you allow the shastras to continue to be',
 'held as sacred and infallible? You must take a stand against the scriptures. Either you must stop',
 'the scriptures. Either you must stop the practice of caste or you must stop believing',
 'in the shastras. You cannot have both. The problem of caste is not a problem of social reform. It',
 'is not a problem of social reform. It is a problem of overthrowing the authority of the',
 'shastras. So long as people believe in the sanctity of the shastras, they will never be able to get',
 'they will never be able to get rid of caste. The work of social reform is like the',
 'work of a gardener who is constantly pruning the leaves and branches of a tree without ever',
 'and branches of a tree without ever attacking the roots. The real enemy is the belief in the',
 'shastras.']

# Embedding fuction

In [39]:
embedding_function = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2'
)


# VectorDB (ChromaDB)

In [40]:
vector_store = Chroma(
    collection_name="Ambedkar",
    embedding_function=embedding_function,
    persist_directory="./chroma_langchain_db",
)
vector_store.add_texts(texts)

['b5719f43-05ce-46f5-b0d1-9595731e1ee0',
 'bbed9b16-22c0-4b6f-a10b-240bd1013715',
 'b67cdebe-57ac-4d98-a68c-cc0c1ee812ec',
 '154a995b-42d7-4248-97ce-d925fac3097e',
 '7cd84c90-e96f-4e19-bfa6-fb1d9b1870e7',
 'd106c727-98f3-4801-bbf4-0b1eefc5a19c',
 '0dcc656a-b5a8-419c-8127-82da42afa700',
 '0eeb4cba-76d7-4825-8d57-b44ef772730b',
 '63d58af9-4e52-4138-af4e-4661190d3ae5',
 '594e0983-7f8b-482b-94b3-4436f3827051',
 'bde30ec7-691f-4dd3-a3a5-6114adecba8f']

# RetrievalQA 

In [41]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# Chains

In [42]:
prompt = ChatPromptTemplate.from_template("""
Use ONLY the context below to answer the question.

Context:
{context}

Question:
{question}

Answer:
""")

In [43]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [44]:
rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": retriever | format_docs,
    }
    | prompt
    | llm
    | StrOutputParser()
)

## Results

In [48]:
questions = [
    "What is the real remedy for caste system?",
    "What is the real enemy?",
    "Why does social reform fail?",
    "What metaphor is used to describe social reform?",
]

for q in questions:
    print(f"\n{'='*60}")
    print(f"QUESTION: {q}")
    print('='*60)
    answer = rag_chain.invoke(q)
    docs = retriever.invoke(q)
    
    print("ANSWER:\n", answer)
    print("\nRETRIEVED CHUNKS:")
    for i, d in enumerate(docs, 1):
        print(f"{i}. {d.page_content[:100]}...")


QUESTION: What is the real remedy for caste system?
ANSWER:
  According to the provided context, the text does not suggest a solution or a remedy for the caste system. However, it implies that the caste problem might not be resolved through social reform as suggested by some. It seems to hint at an interpretation where the caste system is deeply rooted in ancient texts (the shastras), and thus, changing it may involve a different approach or understanding.

RETRIEVED CHUNKS:
1. in the shastras. You cannot have both. The problem of caste is not a problem of social reform. It...
2. in the shastras. You cannot have both. The problem of caste is not a problem of social reform. It...
3. in the shastras. You cannot have both. The problem of caste is not a problem of social reform. It...

QUESTION: What is the real enemy?
ANSWER:
  The real enemy, as stated in the context multiple times, is "the belief." However, without further information, it's not clear what specific belief is being refer